In [1]:
import brightway2 as bw

# import bw2io
import productivity_boosters as prbs

# from project_path import ROOT_DIR
from pprint import pprint
import pandas as pd

from dataclasses import dataclass

# import bw2data
import bw2calc as bc

from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
import bw2analyzer as ba

In [2]:
bw.projects
bw.projects.set_current("GAC_project")
bw.databases

Databases dictionary with 3 object(s):
	GAC_project_db
	biosphere3
	ei39cutoff

In [3]:
ipcc_methods = [
    met
    for met in bw.methods
    if "ipcc 2021" in str(met).lower()
    and "climate change: including SLCFs" in str(met)
    and "no LT" not in str(met)
    and "GWP100" in str(met)
    or ("ipcc 2013" in str(met).lower() and "no LT" not in str(met) and "GWP100" in str(met))
    # or ("ipcc 2021" in str(met).lower() and "no LT" not in str(met) and "GWP100" in str(met))
][1]# [:2]
ipcc_methods


('IPCC 2021',
 'climate change: including SLCFs',
 'global warming potential (GWP100)')

In [4]:
DB_NAME = "GAC_project_db"

In [15]:
act = [
    ds
    for ds in bw.Database(DB_NAME)
    # if "Cu/PCN" in ds["name"]
    # if "Cu2O/Altman-Buchwald-ligand" in ds["name"]
    # if "1-(naphthalen-1-yl)-1H-imidazole" in ds["name"] and "Cu/PCN" in ds["name"]
    if "1-(naphthalen-1-yl)-1H-imidazole" in ds["name"]
    and "Cu2O/Altman-Buchwald-ligand" in ds["name"]
][0]
act

'1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Altman-Buchwald-ligand catalytic complex' (kilogram, GLO, None)

In [ ]:
# bw.Database(DB_NAME).random()
#TODO should be accepting `methods=str`. Correct

# prbs.pretty_lca(
#     FUs=[{act: 1}],
#     methods=ipcc_methods,
# )

## Print before Graph Traversal

In [5]:
def print_recursive_calculation(activity, lcia_method, lca_obj=None, total_score=None, amount=1, level=0, max_level=3, cutoff=1e-2):
    """"Taken from https://github.com/brightway-lca/brightway2/blob/master/notebooks/Contribution%20analysis%20and%20comparison.ipynb"""
    if lca_obj is None:
        lca_obj = bc.LCA({activity: amount}, lcia_method)
        lca_obj.lci()
        lca_obj.lcia()
        total_score = lca_obj.score
    elif total_score is None:
        raise ValueError
    else:
        lca_obj.redo_lcia({activity: amount})
        if abs(lca_obj.score) <= abs(total_score * cutoff):
            return
    print("{}{:4.3f} ({:06.4f}): {:.70}".format("  " * level, lca_obj.score / total_score, lca_obj.score, str(activity)))
    if level < max_level:
        for exc in activity.technosphere():
            print_recursive_calculation(
                activity=exc.input, 
                lcia_method=lcia_method, 
                lca_obj=lca_obj, 
                total_score=total_score, 
                amount=amount * exc['amount'], 
                level=level + 1, 
                max_level=max_level, 
                cutoff=cutoff
            )    

In [16]:
print_recursive_calculation(activity=act, lcia_method=ipcc_methods)

1.000 (70.3327): '1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Alt
  0.028 (2.0021): '1-iodonaphthalene production' (kilogram, GLO, None)
    0.028 (2.0021): 'market for naphthalene sulfonic acid' (kilogram, GLO, None)
      0.019 (1.3523): 'naphthalene sulfonic acid production' (kilogram, RoW, None)
  0.036 (2.5268): 'market for imidazole' (kilogram, GLO, None)
    0.026 (1.8289): 'imidazole production' (kilogram, RoW, None)
      0.013 (0.8888): 'market for glyoxal' (kilogram, RoW, None)
  0.771 (54.2100): 'Cu2O/Altman-Buchwald-ligand catalytic complex production' (kilogram, 
    0.767 (53.9492): '4,7-Dimethoxy-1,10-phenanthroline (Altman-Buchwald ligand) production
      0.011 (0.8021): 'market for acetic acid, without water, in 98% solution state' (kilogr
      0.012 (0.8247): 'market for acetic acid, without water, in 98% solution state' (kilogr
      0.544 (38.2841): 'market for trichloromethane' (kilogram, RER, None)
      0.016 (1.1572): 'market for aniline

## Graph Traversal

### Documentation regarding Graph Traversal

```python
t = bw.GraphTraversal().calculate(demand={act:1}, method=ipcc_methods[1])
t.keys()
>>> dict_keys(['nodes', 'edges', 'lca', 'counter'])
# ---
t["nodes"][-1]
>>>  {'amount': 1, 'cum': 9.327382788679612, 'ind': 9.327382788679611e-06}`
```

from [bw documentation](https://2.docs.brightway.dev/lca.html#graph-traversal):  
`t["nodes"]` - dictionary (each node = row index in `A` matrix)
- `amount`: The total amount of this node needed to produce the functional unit
- `cum`: The cumulative LCA impact score attributable to the needed amount of this node, including its specific supply chain.
- `ind`: The individual LCA impact score directly attributable to one unit of this node, i.e. the score from the direct emissions and resource consumption of this node.

```python
t["edges"][0]
>>> {'to': -1,
 'from': 21263,
 'amount': 1,
 'exc_amount': 1,
 'impact': 9.327382788679609}
```

from [bw documentation](https://2.docs.brightway.dev/lca.html#graph-traversal):  
``t["edges"]`` - list  
- ``to``: The row index of the node consuming the product.
- ``from``: The row index of the node producing the product.
- ``amount``: The total amount of product from needed for the amount of to needed.
- ``exc_amount``: The amount of from needed for one unit of to, i.e. the value given in the technosphere matrix.
- ``impact``: The total LCA impact score embodied in this edge, i.e. the individual score of from times amount.

In [17]:
t = bw.GraphTraversal().calculate(demand={act: 1}, method=ipcc_methods, cutoff=0.07)
print(f"Calculated {t['counter']} LCAs.")


Calculated 264 LCAs.


In [ ]:
# type(t)
# t.keys()
# t["edges"][4]
# t["nodes"][15996]

# t["nodes"][3172]
# 2.82*0.51133
# t["edges"][2]
# t["nodes"].keys()
# t["nodes"][3047]

In [18]:
rev_activity_dict, _, _ = t["lca"].reverse_dict()

In [19]:
named_edges = []
for edge in t["edges"]:
    named_edge = {"impact": edge["impact"]}
      
    act_from = bw.get_activity(rev_activity_dict[edge["from"]])
    
    named_edge["from"] = act_from["name"] + " (" + act_from["location"] + ")"
    if edge["to"] == -1:
        named_edge["to"] = act["name"]
    else:
        act_to = bw.get_activity(rev_activity_dict[edge["to"]]) 
        named_edge["to"] = act_to["name"] + " (" + act_to["location"]  + ")"
    named_edges.append(named_edge)

In [ ]:
un = ba.GTManipulator().unroll_graph(
    nodes=t["nodes"], edges=t["edges"], score=t["nodes"][-1]["cum"]
)

# un[0].keys()
# un[2]

In [20]:
df_gt = pd.DataFrame(named_edges) #t["edges"])
df_gt = df_gt[["to", "from", "impact"]].rename(columns={"to": "target", "from": "source", "impact": "value"})
# df_gt

## Plot results

In [21]:
links = df_gt.to_dict("records")
links


[{'target': '1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Altman-Buchwald-ligand catalytic complex',
  'source': '1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Altman-Buchwald-ligand catalytic complex (GLO)',
  'value': 70.33267325953112},
 {'target': '1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Altman-Buchwald-ligand catalytic complex (GLO)',
  'source': 'market for N-methyl-2-pyrrolidone (GLO)',
  'value': 11.593745261276785},
 {'target': '1-(naphthalen-1-yl)-1H-imidazole production with homogeneous Cu2O/Altman-Buchwald-ligand catalytic complex (GLO)',
  'source': 'Cu2O/Altman-Buchwald-ligand catalytic complex production (GLO)',
  'value': 54.21002041259485},
 {'target': 'Cu2O/Altman-Buchwald-ligand catalytic complex production (GLO)',
  'source': '4,7-Dimethoxy-1,10-phenanthroline (Altman-Buchwald ligand) production (GLO)',
  'value': 53.94924772053695},
 {'target': '4,7-Dimethoxy-1,10-phenanthroline (Altman-Buchwald lig

In [22]:
# Layout size in mm
mm_to_px = 3.7795275591

w_h_in_mm = (180, 55)
(w_px, h_px) = str(w_h_in_mm[0]*mm_to_px), str(w_h_in_mm[1]*mm_to_px)
(w_px, h_px)

('680.314960638', '207.8740157505')

In [23]:
layout = Layout(width=w_px, height=h_px)
SankeyWidget(
    links=links,
    linkLabelFormat='.1f',
    layout=layout,
    align_link_types=True,
    # margins=dict(top=10, bottom=0, left=250, right=300),
).auto_save_svg('../data/interim/test.svg')

SankeyWidget(align_link_types=True, layout=Layout(height='207.8740157505', width='680.314960638'), linkLabelFo…